# Multi-dimensional arrays

When we say Scientific Computing, manipulating vectors, matrices, tensors and higher dimensional arrays immediately come to mind.
An array is a collection of objects stored in a multi-dimensional grid.
In describing an array, we speak of:

- shape of the array
    - number of dimensions (1-D vector, 2-D matrix, 3-D tensor,...)
    - size along each dimention


- element type
    - an array can hold any type, you can have a vector of vectors or a matrix of tensors, in this sense arrays are nestable
    - scientific computing frequently use Float64 and Int32

## Array inspection

We can use the following functions to understand the structure of an array:

Function | Description
-----|--------------
ndims(A) | the number of dimensions of A
size(A) | a tuple of the dimensions of A, length is ndims()
size(A,n) | the size of A along dimension n
axes(A) | a tuple containing the valid indices of A
axes(A,n) | a range expressing the valid indices along dimension n
length(A) | the number of elements in A
eltype(A) | the type of the elements contained in A
eachindex(A) | an efficient iterator for visiting each position in A
strides(A) | a tuple of the strides in each dimension
stride(A,k) | the stride (linear index distance between adjacent elements) along dimension k

## Examples

Here's a **vector**:

In [1]:
v = [1, 2, 3, 4]

4-element Array{Int64,1}:
 1
 2
 3
 4

"Array{Int64,1}" says it is a 1 dimensional array containg Int64 elements, "12-element" says it has 12 elements.

In [2]:
typeof(v)             # typeof() says it is an array

Array{Int64,1}

In [3]:
ndims(v)

1

In [4]:
length(v)

4

In [5]:
eltype(v)

Int64

Here's a **matrix**:

In [6]:
matrix = reshape(collect(1:12), 4, 3)        # convert 1:12 to a 4x3 matrix

4×3 Array{Int64,2}:
 1  5   9
 2  6  10
 3  7  11
 4  8  12

Julia takes the elements and goes down the column (first index) first.

In [7]:
typeof(matrix)

Array{Int64,2}

In [8]:
ndims(matrix)

2

In [9]:
size(matrix)

(4, 3)

In [10]:
length(matrix)

12

In [11]:
eltype(matrix)

Int64

Here's a **tensor**:

In [12]:
tensor = reshape(collect(1.:24.), 2, 3, 4)        # 2 x 3 x 4 tensor

2×3×4 Array{Float64,3}:
[:, :, 1] =
 1.0  3.0  5.0
 2.0  4.0  6.0

[:, :, 2] =
 7.0   9.0  11.0
 8.0  10.0  12.0

[:, :, 3] =
 13.0  15.0  17.0
 14.0  16.0  18.0

[:, :, 4] =
 19.0  21.0  23.0
 20.0  22.0  24.0

In [13]:
typeof(tensor)

Array{Float64,3}

In [14]:
ndims(tensor)          # sometimes called the rank 

3

In [15]:
size(tensor)

(2, 3, 4)

In [16]:
length(tensor)

24

In [17]:
eltype(tensor)        # element type

Float64

What about a **scalar**?

In [18]:
scalar = 2pi

6.283185307179586

In [19]:
typeof(scalar)

Float64

In [20]:
ndims(scalar)         # no dimensions

0

In [21]:
size(scalar)          # size of dimensions is empty

()

In [22]:
length(scalar)        # scalar is length 1

1

In [23]:
eltype(scalar)        # for scalar

Float64

A lot of the errors and initial frustration in array operations stems from shape mis-match.
Understanding the structure of arrays is crucial in writing and debugging scientific programs.

## Element ordering

Julia's array is **column major**, meaning columns are stored contiguously, or more generally, the first index moves the fastest; this is also how FORTRAN stores arrays.
In contrast, C is **row major**, the row is stored contiguously, the last index moves the fastest.
Contiguous memory access is fast as they are likely to be in processor cache rather than RAM.

If performance is critical, you need to pay attention to how you layout an array in memory.
The internal layout also matters if you need to pass arrays to FORTRAN or C. Julia's array is identical to FORTRAN, and C may need the array to be re-arranged.

Julia's built-in array is dense array, elements are layed out contiguously in memory.
There are packages for other types of arrays such as sparse arrays where only non-zero elements are stored.

## Creating an array

There are numerous ways of creating an array:

- from values and shape
- create array from shape
- mimic or copy another array
- by formula or index

### Concatenation

If you know the values of the array, the easiest way to create the array is to use the `[ ]` square bracket notation with different punctuations:

- `[a, b, c, d]` commas denote a 1-D array or Vector, no concatenation happens
- `[a b c d]` spaces denote horizontal concatenation
- `[a; b; c; d]` semicolon or newline denote vertical concatenation

The element type is the smallest type that can store all of the elements.

In [24]:
v = [1, 2, 3, 4]          # , is syntax for 1-D vector

4-element Array{Int64,1}:
 1
 2
 3
 4

In [25]:
m = [1 2 3 4]             # horizontal concatenate of scalars, 2-D matrix

1×4 Array{Int64,2}:
 1  2  3  4

In [26]:
m = [1; 2; 3; 4]          # vertical concatenate of scalars, 1-D vector

4-element Array{Int64,1}:
 1
 2
 3
 4

In [1]:
m = [
    1 2
    3 4
    5 6
]            # vertical concatenate 

3×2 Array{Int64,2}:
 1  2
 3  4
 5  6

In [2]:
m = [1 2; 3 4; 5 6]            # use ; instead of newline

3×2 Array{Int64,2}:
 1  2
 3  4
 5  6

If you want an empty vector (that you can append to later):

In [28]:
[]                        # empty vector

0-element Array{Any,1}

You can specify element type by prefixing the square bracket with the type you want:

In [29]:
Float32[1, 2, 3, 4]

4-element Array{Float32,1}:
 1.0
 2.0
 3.0
 4.0

These convenient `[ ]` syntaxes are syntactic sugar for the `cat()` function.

Syntax | Function | Description
-------|----------|-------------
[A; B; C; ...] | vcat | shorthand for `cat(A...; dims=1)`
[A B C ...] | hcat | shorthand for `cat(A...; dims=2)`
[A B; C D; ...] | hvcat | simultaneous vertical and horizontal concatenation

### Vector concatenation

Because vectors are one-dimentional, we can't really speak of horizontal and vertical directions, this causes issues with vcat and hcat.
Because Julia stores n-dimensional arrays in column major order, it helps to think of vectors as columns when it comes to concatenation.
Therefore, vertical concatenation of vectors results in a longer 1-D vector, and horizontal concatenation results in a matrix. 

What happens internally during concatenation is that all elements are appended serially.
vcat interprets the result as a 1-D vector with length of sum of the individual vector lengths which can be different.
hcat interprets the result as a 2-D matrix with the shape of (length of vector, number of vector), the vectors must all be of the same length.

Suppose we have two vectors as follows:

In [1]:
v1 = [1, 2, 3, 4, 5]

5-element Array{Int64,1}:
 1
 2
 3
 4
 5

In [4]:
v2 = [5, 4, 3, 2, 1]

5-element Array{Int64,1}:
 5
 4
 3
 2
 1

In [6]:
[v1; v2]                # vcat - think of ; as newline

10-element Array{Int64,1}:
 1
 2
 3
 4
 5
 5
 4
 3
 2
 1

In [6]:
[v1 v2 v1 v2]            # hcat

5×4 Array{Int64,2}:
 1  5  1  5
 2  4  2  4
 3  3  3  3
 4  2  4  2
 5  1  5  1

Remember the , is an array constructor, it separates elements of the array,

In [8]:
[v1, v2]        # 2 element array, each of the element is an array; a vector of vectors

2-element Array{Array{Int64,1},1}:
 [1, 2, 3, 4, 5]
 [5, 4, 3, 2, 1]

### Shape specification

When you know the shape of the array, use the `Array()` function:

1. `Array{T}(init, dims...)`

2. `Array{T,N}(init, dims...)`, `Vector{T}` is alias for `Array{T,1}`, `Matrix{T}` is alias for `Array{T,2}`

where:

- **{T}** and **{T,N}** are optional

- **T** is the type such as Float64 or Int

- **N** optional, is the number of dimensions (rank) of the array
    - if not specified, it is length(dims...)
    - if it is specified, it must agree with length(dims...)


- **init** is an initializer
    - **undef** for uninitialized elements (whatever is currently in the allocated memory)
    - **nothing** initialize with nothing, T of Union{Nothing, xxx} is required
    - **missing** for initialization with missing type, T of Union{Missing, xxx} is required
    
    
- **dims...** is the size of each dimension of the array, it can be a comma separated sequence of integers or a tuple of integers

You can also use these functions:

3. **`zeros(T, dims...)`** for an array of 0's

4. **`ones(T, dims...)`** for an array of 1's

5. **`fill(x, dims...)`** for an array filled with x

6. **`trues(dims...)`** for a BitArray of trues

7. **`falses(dims...)`** for a BitArray of falses

8. **`rand(T, dims...)`** for an array filled with random numbers in the range \[0, 1)

9. **`randn(T, dims...)`** for an array filled with standard normal random numbers

10. **`Matrix{T}(I, m, n)`** for identity matrix (needs the *LinearAlgebra* package)

**T** is optional.

Other than Data Science and Statistics, it is unusual for arrays to have missing values.
If your array has missing's, because Missing is a type not a value, you need to explicit specify a Union type,
e.g., if your array elements are Float64 and may be missing, the type is `Union{Missing, Float64}`.
If you want to use nothing (which will cause program to stop), you can use `Union{nothing, Float64}`.

Julia is built for performance, it generates very efficient arithmetic code for arrays.
Missing value checking before arithmetic, or execute and catch condition, incurs a heavy performance penalty in array processing.
By telling Julia whether a variable may have missing value via its type, Julia knows whether to use fast arithmetic code or the slower code with missing handling.
Languages that are not type aware will have a performance penalty somewhere.
Julia's decision to implement missing as a type rather than as a special value is deliberate based on performance considerations.

In [30]:
v = Vector{Union{Missing, Int64}}(missing, 5)         # 5 element vector of Int64/missing with missing

5-element Array{Union{Missing, Int64},1}:
 missing
 missing
 missing
 missing
 missing

In [31]:
m = zeros(4, 4)         # 4x4 matrix of 0's

4×4 Array{Float64,2}:
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0

### Another array

Variables are just names bound to values, thus the assignment operator is really a binding operator, it binds the RHS (right hand side) value to the LHS (left hand side) name.
This avoids copying the RHS object (expensive), the LHS name is merely set to point to the RHS object (fast), this is what all dynamic languages do.
When you put an array on the RHS, what happens is the LHS name becomes a new binding to the same RHS object, the object is not copied.

You can obtain the address of an object with `objectid`:

In [32]:
a = [1, 2, 3]
print(objectid(a))

1856087536225187956

In [33]:
b = a
b === a          # object identity check

true

In [34]:
print(objectid(a), "   ", objectid(b))

1856087536225187956   1856087536225187956

The two names are bound to the same object, they are synonymous.
You can update the object with either name:

In [35]:
a[2] = 5
b[3] = 7
a

3-element Array{Int64,1}:
 1
 5
 7

In [36]:
b

3-element Array{Int64,1}:
 1
 5
 7

This is a large source of error for beginners.  We tend to think of different names as different objects but assignment doesn't copy, you think you are manipulating two variables, but they are in fact the same object, the program will then behave differently from what you expect and the source of error is not apparent and difficult to find.
When you have multiple names pointing to the same object, the program becomes hard to reason about.

If you want another copy of the object, you need to explicitly copy it with **`copy()`**:

In [37]:
b = copy(a)
print(objectid(a), "   ", objectid(b))

1856087536225187956   14828956092135537553

Now a and b are two separate objects.

`copy` is what's known as a shallow copy, it copies the outer structure but not the inner elements (remember arrays can hold anything including other arrays).
If your array elements are numeric primitives, copy() is fine.
But if your array elements are not primitive types, you need to use `deepcopy` which recursively copies everything giving a fully independent object.

You can also alter the shape of an array but keep the underlying data unchanged with `reshape(A, dims...)`:

In [38]:
a = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]

12-element Array{Int64,1}:
  1
  2
  3
  4
  5
  6
  7
  8
  9
 10
 11
 12

In [39]:
reshape(a, 2, 6)

2×6 Array{Int64,2}:
 1  3  5  7   9  11
 2  4  6  8  10  12

In [40]:
reshape(a, 4, 3)

4×3 Array{Int64,2}:
 1  5   9
 2  6  10
 3  7  11
 4  8  12

In [41]:
reshape(a, 2, 2, 3)

2×2×3 Array{Int64,3}:
[:, :, 1] =
 1  3
 2  4

[:, :, 2] =
 5  7
 6  8

[:, :, 3] =
  9  11
 10  12

You can also make another similar array that's uninitialized and optionally change element type and reshape with `similar(A, T, dims)`:

In [42]:
a = [1, 2, 3, 4]
similar(a)

4-element Array{Int64,1}:
 174426112
 164509872
 355845344
 163389984

In [43]:
similar(a, Float64)

4-element Array{Float64,1}:
 1.73835606e-315
 1.73835622e-315
 1.738333137e-315
 2.121995792e-314

In [44]:
similar(a, 2, 2)

2×2 Array{Int64,2}:
 281474976710656  0
      4294967296  0

And you can fill an existing array with another value with `fill!(A, x)`.
Notice the `!` which is a Julia convention to say you are mutating or changing the arguments.
Often it is preferable to update an object in place rather than always returning a new updated copy.

In [45]:
fill!(a, 8)

4-element Array{Int64,1}:
 8
 8
 8
 8

### Comprehension

Comprehension is a concise way to construct an array using an expression, the syntax is more compact than an equivalent do-loop:

In [46]:
[x^2 for x=1:10]

10-element Array{Int64,1}:
   1
   4
   9
  16
  25
  36
  49
  64
  81
 100

If you use multiple index expression separated by comma's, it will create n-D array:

In [47]:
Int32[x * y for x in 1:10, y in 1:10]        # multiplication table

10×10 Array{Int32,2}:
  1   2   3   4   5   6   7   8   9   10
  2   4   6   8  10  12  14  16  18   20
  3   6   9  12  15  18  21  24  27   30
  4   8  12  16  20  24  28  32  36   40
  5  10  15  20  25  30  35  40  45   50
  6  12  18  24  30  36  42  48  54   60
  7  14  21  28  35  42  49  56  63   70
  8  16  24  32  40  48  56  64  72   80
  9  18  27  36  45  54  63  72  81   90
 10  20  30  40  50  60  70  80  90  100

In [48]:
Int32[x % y for x in 1:10, y in 1:10]        # remainder table

10×10 Array{Int32,2}:
 0  1  1  1  1  1  1  1  1  1
 0  0  2  2  2  2  2  2  2  2
 0  1  0  3  3  3  3  3  3  3
 0  0  1  0  4  4  4  4  4  4
 0  1  2  1  0  5  5  5  5  5
 0  0  0  2  1  0  6  6  6  6
 0  1  1  3  2  1  0  7  7  7
 0  0  2  0  3  2  1  0  8  8
 0  1  0  1  4  3  2  1  0  9
 0  0  1  2  0  4  3  2  1  0

The dimension is governed by the for indices, the element type is governed by the formula ahead of for.
If instead, you use multiple for, the result is serialized:

In [49]:
[(x, y) for x in 1:4 for y in 1:3]          # like nested for loop

12-element Array{Tuple{Int64,Int64},1}:
 (1, 1)
 (1, 2)
 (1, 3)
 (2, 1)
 (2, 2)
 (2, 3)
 (3, 1)
 (3, 2)
 (3, 3)
 (4, 1)
 (4, 2)
 (4, 3)

You can even filter the results with an additional if:

In [50]:
[(x, y) for x in 1:4 for y in 1:3 if x+y >= 5]

6-element Array{Tuple{Int64,Int64},1}:
 (2, 3)
 (3, 2)
 (3, 3)
 (4, 1)
 (4, 2)
 (4, 3)

### Generator

The creation methods above instantiate the array in its entirety in memory.
There may be situations where you merely want to pass on a temporary array to some other function to process.
You can create a generator simply by removing the enclosing square brackets.
When used as an argument in a function, it will iterate and produce values on demand instead of producing the entire array. 

In [51]:
sum(1/n^2 for n in 1:10)

1.5497677311665408

In [52]:
sum(1/n^2 for n in 1:1_000)

1.6439345666815615

In [53]:
sum(1/n^2 for n in 1:1_000_000)             # Basel number, π²/6, solved by Euler in 1734

1.64493306684877

In [54]:
using BenchmarkTools
@btime sum(1/n^2 for n in 1:1_000_000)      # Basel number, π²/6, solved by Euler in 1734

  1.390 ms (0 allocations: 0 bytes)


1.64493306684877

In [55]:
π^2 / 6.

1.6449340668482264

## Indexing

Julia is very flexible in indexing to support a wide variety of needs.
Indexed array expression can read values, and if placed on the LHS of assignment operator, those indexed values can be over-written.
The general index form is you provide an expression for each dimension, some commonly used expression are:

- A scalar index, an integer

- A collection of scalars such as:
    - a range like 1:10
    - a vector of scalars


- `:` to select all indices for that dimension

- a boolean array to select true index positions

You can also index a multi-dimensional array with a single index, this ignores the multi-dimensionality and treats the array as a vector and indexes by position, this is called **Linear Indexing**.

In [2]:
a = reshape(collect(1:12), 4, 3)

4×3 Array{Int64,2}:
 1  5   9
 2  6  10
 3  7  11
 4  8  12

Linear indexing a matrix:

In [3]:
a[4]                              # Linear indexing with a scalar

4

In [4]:
a[[1, 4, 9]]                      # Linear indexing with a vector

3-element Array{Int64,1}:
 1
 4
 9

In [5]:
a[1, 1], a[2, 2], a[3, 3]          # scalar for each dimention

(1, 6, 11)

In [6]:
a[:, 2]                            # 2nd column

4-element Array{Int64,1}:
 5
 6
 7
 8

In [7]:
a[3, :]                            # 3rd row

3-element Array{Int64,1}:
  3
  7
 11

In [8]:
a[[1, 3], [1, 3]]                  # a vector for each dimention

2×2 Array{Int64,2}:
 1   9
 3  11

In [10]:
a[[true, false, true, true], [1, 3]]     # boolean mask

3×2 Array{Int64,2}:
 1   9
 3  11
 4  12

In [11]:
a[[true, false, true, false], [1, 3]] = [11 33; 22 44]    # assignment
a

4×3 Array{Int64,2}:
 11  5  33
  2  6  10
 22  7  44
  4  8  12

## Iterator

There are two general methods of iterating over all elements of an array, the distinction is whether you want the elements or the indices.
To iterate over elements:

```
for v in array
    # do something with v
end
```

In [12]:
for x in a
    println(x)
end

11
2
22
4
5
6
7
8
33
10
44
12


To iterate over indices:

```
for i in eachindex(A)
    # Do something with i and/or A[i]
end
```

In [13]:
for i in eachindex(a)
    println(i, "   ", a[i])
end

1   11
2   2
3   22
4   4
5   5
6   6
7   7
8   8
9   33
10   10
11   44
12   12


In [17]:
for c in axes(a, 2)                         # for a given column position
    for r in axes(a, 1)                     # change 1st index (row) -- go down a column
        println("$r $c :  $(a[r, c])")
    end
end

1 1 :  11
2 1 :  2
3 1 :  22
4 1 :  4
1 2 :  5
2 2 :  6
3 2 :  7
4 2 :  8
1 3 :  33
2 3 :  10
3 3 :  44
4 3 :  12
